In [1]:
import pandas as pd
from pysus.preprocessing.geodata import add_data_to_municipality
from pysus.preprocessing.SIM import group_and_count, redistribute_missing, redistribute_cid_chapter
from pysus.preprocessing.decoders import translate_variables_SIM

from pysus.ftp.databases.sim import SIM

sim = SIM().load()

states = ["AC", "AM", "AP", "PA", "RO", "RR", "TO"]
variables = ['CODMUNRES','SEXO','IDADE_ANOS','CID10_CHAPTER']
filter_variables = variables[:3]
geo_df = pd.DataFrame()
ages_args = {
    "start": 0,
    "end": 90,
    "freq": 5
}

In [2]:
import warnings
warnings.filterwarnings('ignore')

for state in states:
    df = sim.download(sim.get_files("CID10",state,2018)).to_dataframe()
    df = translate_variables_SIM(df,age_classes=True,classify_args=ages_args,classify_cid10_chapters=True)
    df = df[variables]
    counts = group_and_count(df,variables)
    counts = redistribute_missing(counts,filter_variables)
    counts = redistribute_cid_chapter(counts,filter_variables)
    geo_df = geo_df.append(add_data_to_municipality(counts,title_cols=['SEXO','IDADE_ANOS','CID10_CHAPTER'], nan_string='NA'))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 330k/330k [00:00<00:00, 167MB/s]
2023-09-19 13:53:06.107 | DEBUG    | pysus.online_data.SIM:get_municipios:185 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2023-09-19 13:53:06.128 | DEBUG    | pysus.online_data.SIM:get_municipios:189 - Changing FTP work dir to: /dissemin/publicos/SIM/CID10/TABELAS
2023-09-19 13:53:06.129 | INFO     | pysus.online_data.SIM:get_municipios:199 - Local parquet file found at /home/bida/pysus/SIM_CADMUN_.parquet
2023-09-19 13:53:06.371 | DEBUG    | pysus.online_data.SIM:get_CID10_chapters_table:47 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2023-09-19 13:53:06.392 | DEBUG    | pysus.online_data.SIM:get_CID10_chapters_table:51 - Changing FTP work dir to: /dissemin/publicos/SIM/CID10/TABELAS
2023-09-19 13:53:06.393 | INFO     | pysus.online_data.SIM:get_CID10_chapters_table:61 - Local parque

In [3]:
geo_df.columns

Index(['code_muni', 'name_muni', 'code_state', 'abbrev_state', 'name_state',
       'code_region', 'name_region', 'geometry', 'Feminino-[0.0, 5.0)-1',
       'Feminino-[0.0, 5.0)-2',
       ...
       'NA-[50.0, 55.0)-8', 'NA-[55.0, 60.0)-8', 'NA-[60.0, 65.0)-8',
       'NA-[65.0, 70.0)-8', 'NA-[70.0, 75.0)-8', 'NA-[75.0, 80.0)-8',
       'NA-[80.0, 85.0)-8', 'NA-[85.0, 90.0)-8', 'NA-[90.0, inf)-8',
       'NA-NA-8'],
      dtype='object', length=1088)